# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f213c9a2c70>
├── 'a' --> tensor([[-0.1007,  1.4888, -0.1834],
│                   [-1.1105, -1.5042,  0.1515]])
└── 'x' --> <FastTreeValue 0x7f2140a91790>
    └── 'c' --> tensor([[-0.6514,  0.2834,  0.9189,  0.1129],
                        [-2.0834, -0.0438,  1.8263, -0.8225],
                        [ 0.8366, -1.6965, -1.3708,  1.6592]])

In [4]:
t.a

tensor([[-0.1007,  1.4888, -0.1834],
        [-1.1105, -1.5042,  0.1515]])

In [5]:
%timeit t.a

63.5 ns ± 0.103 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f213c9a2c70>
├── 'a' --> tensor([[-1.1484, -0.2408, -0.4860],
│                   [-0.3185, -1.5533, -1.9912]])
└── 'x' --> <FastTreeValue 0x7f2140a91790>
    └── 'c' --> tensor([[-0.6514,  0.2834,  0.9189,  0.1129],
                        [-2.0834, -0.0438,  1.8263, -0.8225],
                        [ 0.8366, -1.6965, -1.3708,  1.6592]])

In [7]:
%timeit t.a = new_value

76 ns ± 0.143 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1007,  1.4888, -0.1834],
               [-1.1105, -1.5042,  0.1515]]),
    x: Batch(
           c: tensor([[-0.6514,  0.2834,  0.9189,  0.1129],
                      [-2.0834, -0.0438,  1.8263, -0.8225],
                      [ 0.8366, -1.6965, -1.3708,  1.6592]]),
       ),
)

In [10]:
b.a

tensor([[-0.1007,  1.4888, -0.1834],
        [-1.1105, -1.5042,  0.1515]])

In [11]:
%timeit b.a

58.3 ns ± 0.156 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7434, -0.5176, -0.2237],
               [ 0.2344, -0.2908, -0.9556]]),
    x: Batch(
           c: tensor([[-0.6514,  0.2834,  0.9189,  0.1129],
                      [-2.0834, -0.0438,  1.8263, -0.8225],
                      [ 0.8366, -1.6965, -1.3708,  1.6592]]),
       ),
)

In [13]:
%timeit b.a = new_value

494 ns ± 0.0498 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

813 ns ± 0.184 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 18.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 255 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 583 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f213c964040>
├── 'a' --> tensor([[[-0.1007,  1.4888, -0.1834],
│                    [-1.1105, -1.5042,  0.1515]],
│           
│                   [[-0.1007,  1.4888, -0.1834],
│                    [-1.1105, -1.5042,  0.1515]],
│           
│                   [[-0.1007,  1.4888, -0.1834],
│                    [-1.1105, -1.5042,  0.1515]],
│           
│                   [[-0.1007,  1.4888, -0.1834],
│                    [-1.1105, -1.5042,  0.1515]],
│           
│                   [[-0.1007,  1.4888, -0.1834],
│                    [-1.1105, -1.5042,  0.1515]],
│           
│                   [[-0.1007,  1.4888, -0.1834],
│                    [-1.1105, -1.5042,  0.1515]],
│           
│                   [[-0.1007,  1.4888, -0.1834],
│                    [-1.1105, -1.5042,  0.1515]],
│           
│                   [[-0.1007,  1.4888, -0.1834],
│                    [-1.1105, -1.5042,  0.1515]]])
└── 'x' --> <FastTreeValue 0x7f207a85daf0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 70.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2071fa0640>
├── 'a' --> tensor([[-0.1007,  1.4888, -0.1834],
│                   [-1.1105, -1.5042,  0.1515],
│                   [-0.1007,  1.4888, -0.1834],
│                   [-1.1105, -1.5042,  0.1515],
│                   [-0.1007,  1.4888, -0.1834],
│                   [-1.1105, -1.5042,  0.1515],
│                   [-0.1007,  1.4888, -0.1834],
│                   [-1.1105, -1.5042,  0.1515],
│                   [-0.1007,  1.4888, -0.1834],
│                   [-1.1105, -1.5042,  0.1515],
│                   [-0.1007,  1.4888, -0.1834],
│                   [-1.1105, -1.5042,  0.1515],
│                   [-0.1007,  1.4888, -0.1834],
│                   [-1.1105, -1.5042,  0.1515],
│                   [-0.1007,  1.4888, -0.1834],
│                   [-1.1105, -1.5042,  0.1515]])
└── 'x' --> <FastTreeValue 0x7f2071fa0280>
    └── 'c' --> tensor([[-0.6514,  0.2834,  0.9189,  0.1129],
                        [-2.0834, -0.0438,  1.8263, -0.8225],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 108 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

64.8 µs ± 79.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.6514,  0.2834,  0.9189,  0.1129],
                       [-2.0834, -0.0438,  1.8263, -0.8225],
                       [ 0.8366, -1.6965, -1.3708,  1.6592]],
              
                      [[-0.6514,  0.2834,  0.9189,  0.1129],
                       [-2.0834, -0.0438,  1.8263, -0.8225],
                       [ 0.8366, -1.6965, -1.3708,  1.6592]],
              
                      [[-0.6514,  0.2834,  0.9189,  0.1129],
                       [-2.0834, -0.0438,  1.8263, -0.8225],
                       [ 0.8366, -1.6965, -1.3708,  1.6592]],
              
                      [[-0.6514,  0.2834,  0.9189,  0.1129],
                       [-2.0834, -0.0438,  1.8263, -0.8225],
                       [ 0.8366, -1.6965, -1.3708,  1.6592]],
              
                      [[-0.6514,  0.2834,  0.9189,  0.1129],
                       [-2.0834, -0.0438,  1.8263, -0.8225],
                       [ 0.8366, -1.6965, -1.3708,  1.6592]],

In [26]:
%timeit Batch.stack(batches)

78.9 µs ± 142 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.6514,  0.2834,  0.9189,  0.1129],
                      [-2.0834, -0.0438,  1.8263, -0.8225],
                      [ 0.8366, -1.6965, -1.3708,  1.6592],
                      [-0.6514,  0.2834,  0.9189,  0.1129],
                      [-2.0834, -0.0438,  1.8263, -0.8225],
                      [ 0.8366, -1.6965, -1.3708,  1.6592],
                      [-0.6514,  0.2834,  0.9189,  0.1129],
                      [-2.0834, -0.0438,  1.8263, -0.8225],
                      [ 0.8366, -1.6965, -1.3708,  1.6592],
                      [-0.6514,  0.2834,  0.9189,  0.1129],
                      [-2.0834, -0.0438,  1.8263, -0.8225],
                      [ 0.8366, -1.6965, -1.3708,  1.6592],
                      [-0.6514,  0.2834,  0.9189,  0.1129],
                      [-2.0834, -0.0438,  1.8263, -0.8225],
                      [ 0.8366, -1.6965, -1.3708,  1.6592],
                      [-0.6514,  0.2834,  0.9189,  0.1129],
                   

In [28]:
%timeit Batch.cat(batches)

148 µs ± 221 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

338 µs ± 2.25 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
